In [0]:
# needed to import excel files
%pip install -i https://pypi.org/simple openpyxl

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.


In [0]:
from datetime import datetime, timedelta 
from getpass import getuser as get_user 
from getpass import getpass as get_password 
from os import environ as enviroment_vars 
from os import setpgrp as set_process_group 
from random import random 
from shlex import split as cmd_split 
from string import punctuation 
from subprocess import Popen, PIPE, STDOUT 
import pandas as pd 

In [0]:
from pyspark import SparkContext, HiveContext, SparkConf  
from pyspark.sql import SparkSession 

ss = SparkSession(sc) 
ss 
spark.conf.set("spark.databricks.io.cache.enabled", "true")

In [0]:
#Create Integer Date filters for the monthly calculation
from datetime import datetime, timedelta 
from dateutil.relativedelta import relativedelta
today = datetime.today()


CURRENT_YTD_END = datetime(today.year, today.month, 1) - relativedelta(days=1)
CURRENT_YTD_BEG = datetime(today.year, 1, 1)
PRIOR_YTD_END = datetime(today.year - 1, today.month, 1) - relativedelta(days=1)
PRIOR_YTD_BEG = datetime(PRIOR_YTD_END.year, 1, 1)
PRIOR_YTD = datetime(today.year - 1, today.month, 1) - relativedelta(days=1)

#CURRENT_YTD_END = datetime(today.year, today.month - 1, 31) #change date number to last day of the previous month
#CURRENT_YTD_BEG = datetime(today.year, 1, 1)
#Previous year - full year instead of YTD - Feb 2022
#PRIOR_YTD_END = datetime(today.year - 1, today.month - 1, 31) 
#PRIOR_YTD_BEG = datetime(PRIOR_YTD_END.year, 1, 1)
#PRIOR_YTD = datetime(today.year - 1, today.month - 1, 31) #change date number to last day of the previous month

#ONLY USE BELOW FOR JAN EACH YEAE
#CURRENT_YTD_END = int(datetime(today.year-1, 12, 31).strftime('%Y%m%d')) 
#CURRENT_YTD_BEG = int(datetime(today.year-1, 1, 1).strftime('%Y%m%d')) 
#PRIOR_YTD_END = int(datetime(today.year - 2, 12, 31).strftime('%Y%m%d'))  
#PRIOR_YTD_BEG = int(datetime(today.year - 2, 1, 1).strftime('%Y%m%d')) 

print("Current year end: " + str(CURRENT_YTD_END))
print("Current year beginning: " + str(CURRENT_YTD_BEG))
print("Previous year end: " + str(PRIOR_YTD_END))
print("Previous year beginning: " + str(PRIOR_YTD_BEG))
#print("Previous year YTD: " + str(PRIOR_YTD))

Current year end: 2024-08-31 00:00:00
Current year beginning: 2024-01-01 00:00:00
Previous year end: 2023-08-31 00:00:00
Previous year beginning: 2023-01-01 00:00:00


In [0]:
#Data Source: Data Lake - as HBT missing low ratio ever ind field
#Pull in 2 years HO Low Ratio ALL CMHC Purchase and Refinance loans 
#Use low ratio ever ind = 1 to identifiy low ratio ever loans
import pandas as pd 

a = {'CURRENT_YTD_BEG':CURRENT_YTD_BEG, 
     'CURRENT_YTD_END':CURRENT_YTD_END, 
     'PRIOR_YTD_BEG':PRIOR_YTD_BEG,  
     'PRIOR_YTD_END':PRIOR_YTD_END,
     }

sql_lr = """
select distinct lad.cmhc_loan_account_nbr,
lad.application_received_system_dt as received_date,
case 
when lad.loan_purpose_tcd in ('1', '4', '7', '8', '9', 'D', '**') then 'Purchase'       
when lad.loan_purpose_tcd in ('2', '3', '5', 'A', 'E') then 'Refinance'
else 'Unknown'   
end as loan_purpose,

CASE WHEN emili.LNPURPCD IN ('','7','8','9','1','4') THEN 'PURCHASE'  
WHEN emili.LNPURPCD ='D' THEN 'PORT' 
ELSE 'REFINANCE' END AS Loan_Purpose_new,

CASE
WHEN (lad.application_received_system_dt between '{2}' and '{4}') then "PYTD"
WHEN (lad.application_received_system_dt between '{0}' and '{1}') THEN "CYTD"
else "NA"
END AS REC_YEAR,
CASE
WHEN (lad.application_received_system_dt between '{2}' and '{4}') then 1 else 0 
end as PYTD,
CASE
WHEN (lad.application_received_system_dt between '{0}' and '{1}') THEN 1 else 0
END AS CYTD,
las.property_lending_value_amt AS LENDING_VALUE,
las.loan_insured_amt,
las.loan_lender_amt AS LOAN_AMT,
case 
when lad.cmhc_loan_account_nbr is not null then 1 else 0 end as loan_cnt,
lad.low_ratio_insurable_tcd,
case 
when lad.low_ratio_insurable_tcd = '00' then 'Non-insurable'
when lad.low_ratio_insurable_tcd = '01' then 'Insurable'
when lad.low_ratio_insurable_tcd = '02' then 'Insur Via Portfolio'
when lad.low_ratio_insurable_tcd = '**' AND lad.low_ratio_mortgage_tcd = '03' AND t1.account_status_tcd = '06' 
THEN 'Insurable' 
ELSE 'Non-insurable'
END AS INSURABLE_RESPONSE,

lad.low_ratio_mortgage_type_desc_en,
case 
when lad.low_ratio_mortgage_tcd in ('**', '03') then "Insurance" 
else "Low Ratio Assessment" 
end as Insurance,

lft.parent_level1_lender_nm_en AS PARENT_LENDER,
lft.parent_level1_lender_cde,
lft.parent_level2_lender_nm_en,
lft.parent_level2_lender_cde,

trim(lft.transit_id) as transit_id,
trim(lad.contact_local_telephone_area_cde)||""||trim(lad.contact_local_telephone_nbr) AS TELEPHONE,
lad.business_source_identification_cde,
lad.business_source_identification_desc_en,
lad.contact_nm,
concat(trim(lad.original_representative_first_nm), ' ', trim(lad.original_representative_last_nm)) as Original_Rep,
pad.property_address_municipality_nm as PROP_Municipality,
pad.current_stats_canada_province_nm_en as province,
CASE 
WHEN trim(pad.current_stats_canada_province_cde) in ('00', '03', '01', '02') THEN 'ATLANTIC' 
WHEN trim(pad.current_stats_canada_province_cde) in ('08', '06', '07', '11', '13', '12') 
THEN 'PRAIRIES' 
ELSE trim(pad.current_stats_canada_province_nm_en) 
END AS REGION, 
pad.dwelling_type_desc_en AS Dwelling_type, 
pad.tenure_type_desc_en as Tenure_type

from cmwdw_insurance.vwdw_loan_account_dim LAD

left join cmwdw_insurance.vwdw_loan_account_sum LAS
ON (lad.loan_account_surrkey=las.loan_account_surrkey)

left join cmwdw_insurance.vwdw_costable_unit_dim CUD
ON (las.costable_unit_surrkey=CUD.costable_unit_surrkey)

Left join cmwdw_insurance.vwdw_lender_financial_trnst_dim lfT 
ON (las.initiation_lender_financial_transit_surrkey = lfT.lender_financial_transit_surrkey) 

left join cmwdw_insurance.tbdw_tgt_property_account_dim pad 
ON (las.property_account_surrkey=pad.property_account_surrkey)

left join cmwdw_insurance.vwdw_loan_status_dim t1
on (t1.loan_status_surrkey = las.loan_status_surrkey)

left join ds_ins_risk.ins_risk_last emili
on lad.cmhc_loan_account_nbr=emili.lnacctno

WHERE
lad.low_ratio_ever_ind=1
AND lad.application_received_system_dt >='2020-01-01'
AND cud.insurance_product_group_cde = '100' 
"""

df_lr_p = ss.sql(sql_lr.format(a.get('CURRENT_YTD_BEG'), a.get('CURRENT_YTD_END'), a.get('PRIOR_YTD_BEG'), a.get('PRIOR_YTD_END'), a.get('PRIOR_YTD'))).toPandas()

#Calculate Fiscal Year for Received date
df_lr_p['received_date'] = pd.to_datetime(df_lr_p['received_date'])
df_lr_p['Rec_fiscal_year'] = df_lr_p['received_date'].dt.to_period('Q-OCT').dt.qyear

df_lr_p = df_lr_p.dropna(axis='columns', how='all') 
#df_lr_sit = ss.createDataFrame(df_lr_p)
#df_lr_sit.registerTempTable("df_lr_sit")
#print("HO Low Raio Loan Level Base Data row count: ", df_lr_p.count())
print("Dataframe creation for HO Low Raio Loan Level Base Data completed at: ", str(datetime.now()))

Dataframe creation for HO Low Raio Loan Level Base Data completed at:  2024-09-06 18:07:58.814613


distinct_count = df_lr_p["PARENT_LENDER"].nunique()

print("Distinct count:", distinct_count)

In [0]:
#Remove specialt charaters from Contact name and Original Rep fields
df_lr_p = df_lr_p.replace({"Contact_nm":{"-":"", "'":""}, 'Original_Rep':{"-":"", "'":""}})

#Remove space in the Original Rep columns for the joining steps
#df_lr_p['Original_Rep'] = df_lr_p['Original_Rep'].replace(['-', ' '], '', regex=True).str.upper()

In [0]:
#Create 6 new columns with different reordered Contact_nm words to match with BNS Name List
df_lr_p['Contact_1'] = df_lr_p['contact_nm'].astype(str).str.split().str[0].fillna('') + df_lr_p['contact_nm'].astype(str).str.split().str[1].fillna('') + df_lr_p['contact_nm'].astype(str).str.split().str[2].fillna('')
df_lr_p['Contact_2'] = df_lr_p['contact_nm'].astype(str).str.split().str[0].fillna('') + df_lr_p['contact_nm'].astype(str).str.split().str[2].fillna('') + df_lr_p['contact_nm'].astype(str).str.split().str[1].fillna('')
df_lr_p['Contact_3'] = df_lr_p['contact_nm'].astype(str).str.split().str[1].fillna('') + df_lr_p['contact_nm'].astype(str).str.split().str[0].fillna('') + df_lr_p['contact_nm'].astype(str).str.split().str[2].fillna('')
df_lr_p['Contact_4'] = df_lr_p['contact_nm'].astype(str).str.split().str[1].fillna('') + df_lr_p['contact_nm'].astype(str).str.split().str[2].fillna('') + df_lr_p['contact_nm'].astype(str).str.split().str[0].fillna('')
df_lr_p['Contact_5'] = df_lr_p['contact_nm'].astype(str).str.split().str[2].fillna('') + df_lr_p['contact_nm'].astype(str).str.split().str[1].fillna('') + df_lr_p['contact_nm'].astype(str).str.split().str[0].fillna('')
df_lr_p['Contact_6'] = df_lr_p['contact_nm'].astype(str).str.split().str[2].fillna('') + df_lr_p['contact_nm'].astype(str).str.split().str[0].fillna('') + df_lr_p['contact_nm'].astype(str).str.split().str[1].fillna('')

#Create 6 new columns with different reordered Original Rep words to match with BNS Name List
df_lr_p['Rep_1'] = df_lr_p['Original_Rep'].astype(str).str.split().str[0].fillna('') + df_lr_p['Original_Rep'].astype(str).str.split().str[1].fillna('') + df_lr_p['Original_Rep'].astype(str).str.split().str[2].fillna('')
df_lr_p['Rep_2'] = df_lr_p['Original_Rep'].astype(str).str.split().str[0].fillna('') + df_lr_p['Original_Rep'].astype(str).str.split().str[2].fillna('') + df_lr_p['Original_Rep'].astype(str).str.split().str[1].fillna('')
df_lr_p['Rep_3'] = df_lr_p['Original_Rep'].astype(str).str.split().str[1].fillna('') + df_lr_p['Original_Rep'].astype(str).str.split().str[0].fillna('') + df_lr_p['Original_Rep'].astype(str).str.split().str[2].fillna('')
df_lr_p['Rep_4'] = df_lr_p['Original_Rep'].astype(str).str.split().str[1].fillna('') + df_lr_p['Original_Rep'].astype(str).str.split().str[2].fillna('') + df_lr_p['Original_Rep'].astype(str).str.split().str[0].fillna('')
df_lr_p['Rep_5'] = df_lr_p['Original_Rep'].astype(str).str.split().str[2].fillna('') + df_lr_p['Original_Rep'].astype(str).str.split().str[1].fillna('') + df_lr_p['Original_Rep'].astype(str).str.split().str[0].fillna('')
df_lr_p['Rep_6'] = df_lr_p['Original_Rep'].astype(str).str.split().str[2].fillna('') + df_lr_p['Original_Rep'].astype(str).str.split().str[0].fillna('') + df_lr_p['Original_Rep'].astype(str).str.split().str[1].fillna('')



In [0]:
#df_lr_p.columns.duplicated()

In [0]:
df_lr_sit = ss.createDataFrame(df_lr_p) 
df_lr_sit.createOrReplaceTempView("df_lr_sit")
#print("Low Ratio Loan Level Base Data with Updated Contact Name row count: ", df_lr_sit.count())
print("Table creation for Low Ratio Base Data with Updated Contact Name completed at: ", str(datetime.now()))

Table creation for Low Ratio Base Data with Updated Contact Name completed at:  2024-09-06 18:31:25.477090


In [0]:
#Read the BNS Name List Excel File and save as dataframe
from pyspark.sql.functions import trim
bns = pd.read_excel('BNS Name List Last Update.xlsx', dtype = {'INACTIVE_DATE_LASTUPDATE': str}).astype(str)
bns['LENDER_CHANNEL_LASTUPDATE'] = bns['LENDER_CHANNEL_LASTUPDATE'].replace(' ', '', regex=True)
#bns['EFFECTIVE_DATE_LASTUPDATE'] = pd.to_datetime(bns['EFFECTIVE_DATE_LASTUPDATE'], yearfirst=True)
#bns['INACTIVE_DATE_LASTUPDATE'] = pd.to_datetime(bns['INACTIVE_DATE_LASTUPDATE'], yearfirst=True)
df_bns = ss.createDataFrame(bns)
df_bns = df_bns.withColumn("LENDER_CHANNEL_LASTUPDATE", trim(df_bns.LENDER_CHANNEL_LASTUPDATE))
df_bns.createOrReplaceTempView("df_bns")

#Read the HFS Name List Excel File and save as dataframe
hfs = pd.read_excel('HFS Name List Last Update.xlsx', dtype = {'INACTIVE_DATE_LASTUPDATE': str,
                                                              'EFFECTIVE_DATE_LASTUPDATE': str}).astype(str)
#hfs['EFFECTIVE_DATE_LASTUPDATE'] = pd.to_datetime(hfs['EFFECTIVE_DATE_LASTUPDATE'], yearfirst=True)
#hfs['INACTIVE_DATE_LASTUPDATE'] = pd.to_datetime(hfs['INACTIVE_DATE_LASTUPDATE'], yearfirst=True)
df_hfs = ss.createDataFrame(hfs)
df_hfs.createOrReplaceTempView("df_hfs")

In [0]:
#Merge Base Data with BNS Employee Name List with VP, Sales Manager info
sql_a = """
select t1.*,
t2.EMPLOYEENO_LASTUPDATE as RU_EMPLOYEENO, 
t2.NAME_LASTUPDATE as RU_NAME1, 
t2.LENDER_CHANNEL_LASTUPDATE as bns_lender_channel, 
t2.EFFECTIVE_DATE_LASTUPDATE as RU_Effectivedt, 
t2.INACTIVE_DATE_LASTUPDATE as RU_Inactivedt, 
t2.REGIONAL_VP_LASTUPDATE as RU_VP, 
t2.PROVINCE_LASTUPDATE as RU_Province, 
case when t2.EMPLOYEENO_LASTUPDATE is null then 0 else 1 
end as bnsnamelistmatch_ind,
CASE
   WHEN t1.transit_id IN ('01339', '13151', '27912','78386') THEN 'MOSE'
   WHEN t1.transit_id IN ('85712') THEN
                  CASE WHEN business_source_identification_desc_en = 'BROKER' THEN 'MOSE' 
                       WHEN business_source_identification_desc_en = 'MORTGAGE SPECIALIST' 
                       AND t2.LENDER_CHANNEL_LASTUPDATE = 'MOSE' THEN 'MOSE' 
                  ELSE 'HFLM' 
                  END
    WHEN t1.transit_id IN ('77966') THEN 
        CASE WHEN business_source_identification_desc_en = 'MORTGAGE SPECIALIST' THEN 'MOSE'
             WHEN business_source_identification_desc_en NOT IN ('MORTGAGE SPECIALIST') THEN 
                 CASE WHEN t2.LENDER_CHANNEL_LASTUPDATE = 'BRM' THEN 'BRM'
                 ELSE 'MOSE'
                 END
        END
    WHEN t1.transit_id NOT IN ('77966','85712','01339', '13151', '27912','78386') THEN 'BRANCH'
END 
AS lenderchannel
from df_lr_sit t1 left join df_bns t2 on 
((trim(t1.Contact_1) = trim(t2.NAME_LASTUPDATE) or (T2.NAME2_LASTUPDATE IS NOT NULL AND trim(t1.Contact_1) = trim(t2.NAME2_LASTUPDATE)))
and (t1.received_date >= t2.effective_date_lastupdate and (t1.received_date < t2.INACTIVE_DATE_LASTUPDATE OR t2.inactive_date_lastupdate is null))
) 
or 
((trim(t1.Contact_2) = trim(t2.NAME_LASTUPDATE) or (T2.NAME2_LASTUPDATE IS NOT NULL AND trim(t1.Contact_2) = trim(t2.NAME2_LASTUPDATE)))
and (t1.received_date >= t2.effective_date_lastupdate and (t1.received_date < t2.INACTIVE_DATE_LASTUPDATE OR t2.inactive_date_lastupdate is null))
) 
or 
((trim(t1.Contact_3) = trim(t2.NAME_LASTUPDATE) or (T2.NAME2_LASTUPDATE IS NOT NULL AND trim(t1.Contact_3) = trim(t2.NAME2_LASTUPDATE)))
and (t1.received_date >= t2.effective_date_lastupdate and (t1.received_date < t2.INACTIVE_DATE_LASTUPDATE OR t2.inactive_date_lastupdate is null))
) 
or 
((trim(t1.Contact_4) = trim(t2.NAME_LASTUPDATE) or (T2.NAME2_LASTUPDATE IS NOT NULL AND trim(t1.Contact_4) = trim(t2.NAME2_LASTUPDATE)))
and (t1.received_date >= t2.effective_date_lastupdate and (t1.received_date < t2.INACTIVE_DATE_LASTUPDATE OR t2.inactive_date_lastupdate is null))
) or 
((trim(t1.Contact_5) = trim(t2.NAME_LASTUPDATE) or (T2.NAME2_LASTUPDATE IS NOT NULL AND trim(t1.Contact_5) = trim(t2.NAME2_LASTUPDATE)))
and (t1.received_date >= t2.effective_date_lastupdate and (t1.received_date < t2.INACTIVE_DATE_LASTUPDATE OR t2.inactive_date_lastupdate is null))
) or 
((trim(t1.Contact_6) = trim(t2.NAME_LASTUPDATE) or (T2.NAME2_LASTUPDATE IS NOT NULL AND trim(t1.Contact_6) = trim(t2.NAME2_LASTUPDATE)))
and (t1.received_date >= t2.effective_date_lastupdate and (t1.received_date < t2.INACTIVE_DATE_LASTUPDATE OR t2.inactive_date_lastupdate is null))
) 
"""

df_base_bns = ss.sql(sql_a)
df_base_bns.registerTempTable("df_base_bns")
#print("Merged with BNS Employee List table row count: ", df_base_bns.count())
print("Merged with BNS Employee List Dataframe creation completed at: ", str(datetime.now()))

Merged with BNS Employee List Dataframe creation completed at:  2024-09-06 18:31:26.997979


/databricks/spark/python/pyspark/sql/dataframe.py:334: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [0]:
#Merge Previous Datafram with HFS home finance sales Employee Name List with VP, Sales Manager info.
sql_b = """
select t1.cmhc_loan_account_nbr,
t2.EMPLOYEENO_LASTUPDATE as HFS_EMPLOYEENO, 
t2.NAME_LASTUPDATE as HFS_NAME1, 
t2.NAME2_LASTUPDATE as HFS_NAME2,
t2.ROLE_LASTUPDATE as HFS_ROLE, 
t2.EFFECTIVE_DATE_LASTUPDATE as HFS_Effectivedt, 
t2.INACTIVE_DATE_LASTUPDATE as HFS_Inactivedt, 
t2.SUPERVISOR_LASTUPDATE as HFS_Supervisor, 
t2.PROVINCE_LASTUPDATE as HFS_Province,  
t2.ROLE_LASTUPDATE, 
t3.SUPERVISOR_LASTUPDATE as HFS_VP,
case when t2.NAME_LASTUPDATE is not null then 1 else 0
end as hfsnamematch_ind,
case when t3.NAME_LASTUPDATE is not null then 1 else 0
end as vphfsnamematch_ind
from df_lr_sit t1 left join df_hfs t2 on 
((trim(t1.Rep_1) = trim(t2.NAME_LASTUPDATE) or (T2.NAME2_LASTUPDATE IS NOT NULL AND trim(t1.Rep_1) = trim(t2.NAME2_LASTUPDATE)))
and (t1.received_date >= t2.effective_date_lastupdate and (t1.received_date < t2.INACTIVE_DATE_LASTUPDATE OR t2.inactive_date_lastupdate is null))
) 
or 
((trim(t1.Rep_2) = trim(t2.NAME_LASTUPDATE) or (T2.NAME2_LASTUPDATE IS NOT NULL AND trim(t1.Rep_2) = trim(t2.NAME2_LASTUPDATE)))
and (t1.received_date >= t2.effective_date_lastupdate and (t1.received_date < t2.INACTIVE_DATE_LASTUPDATE OR t2.inactive_date_lastupdate is null))
) 
or 
((trim(t1.Rep_3) = trim(t2.NAME_LASTUPDATE) or (T2.NAME2_LASTUPDATE IS NOT NULL AND trim(t1.Rep_3) = trim(t2.NAME2_LASTUPDATE)))
and (t1.received_date >= t2.effective_date_lastupdate and (t1.received_date < t2.INACTIVE_DATE_LASTUPDATE OR t2.inactive_date_lastupdate is null))
) 
or 
((trim(t1.Rep_4) = trim(t2.NAME_LASTUPDATE) or (T2.NAME2_LASTUPDATE IS NOT NULL AND trim(t1.Rep_4) = trim(t2.NAME2_LASTUPDATE)))
and (t1.received_date >= t2.effective_date_lastupdate and (t1.received_date < t2.INACTIVE_DATE_LASTUPDATE OR t2.inactive_date_lastupdate is null))
) or 
((trim(t1.Rep_5) = trim(t2.NAME_LASTUPDATE) or (T2.NAME2_LASTUPDATE IS NOT NULL AND trim(t1.Rep_5) = trim(t2.NAME2_LASTUPDATE)))
and (t1.received_date >= t2.effective_date_lastupdate and (t1.received_date < t2.INACTIVE_DATE_LASTUPDATE OR t2.inactive_date_lastupdate is null))
) or 
((trim(t1.Rep_6) = trim(t2.NAME_LASTUPDATE) or (T2.NAME2_LASTUPDATE IS NOT NULL AND trim(t1.Rep_6) = trim(t2.NAME2_LASTUPDATE)))
and (t1.received_date >= t2.effective_date_lastupdate and (t1.received_date < t2.INACTIVE_DATE_LASTUPDATE OR t2.inactive_date_lastupdate is null))
) 
left join df_hfs t3  
ON (trim(t2.SUPERVISOR_LASTUPDATE) = trim(t3.NAME_LASTUPDATE) or 
(trim(T3.NAME2_LASTUPDATE) IS NOT NULL AND trim(t2.SUPERVISOR_LASTUPDATE) = trim(t3.NAME2_LASTUPDATE))) 
AND
(t1.received_date >= t3.effective_date_lastupdate and 
(t1.received_date < t3.INACTIVE_DATE_LASTUPDATE OR t3.inactive_date_lastupdate is NULL))
"""

df_base_hfs = ss.sql(sql_b)
df_base_hfs.registerTempTable("df_base_hfs")
#print("Merged with HFS Employee List table row count: ", df_base_hfs.count())
print("Merged with HFS Employee List Dataframe creation completed at: ", str(datetime.now()))

Merged with HFS Employee List Dataframe creation completed at:  2024-09-06 18:31:27.180812


/databricks/spark/python/pyspark/sql/dataframe.py:334: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [0]:
#Merge BNS and HFS Name List together
sql_c = """
select t1.*,
t2.HFS_EMPLOYEENO, 
t2.HFS_NAME1, 
t2.HFS_NAME2,
t2.HFS_ROLE, 
t2.HFS_Effectivedt, 
t2.HFS_Inactivedt, 
t2.HFS_Supervisor, 
t2.HFS_Province,  
t2.ROLE_LASTUPDATE as HFS_ROLE_LASTUPDATE, 
t2.HFS_VP
from df_base_bns t1 left join df_base_hfs t2
on t1.cmhc_loan_account_nbr = t2.cmhc_loan_account_nbr
"""

df_lr_all = ss.sql(sql_c)
#Drop the 6 newly created contact name and Original Rep columns
df_lr_all.drop("Contact_1", "Contact_2", "Contact_3","Contact_4","Contact_5","Contact_6", \
               "Rep_1", "Rep_2", "Rep_3", "Rep_4", "Rep_5", "Rep_6")
df_lr_all.registerTempTable("df_lr_all")
#print("Low Ratio table with BNS and HFS Employee List table row count: ", df_lr_all.count())
print("Low Ratio Table with BNS and HFS Employee List dataframe creation completed at: ", str(datetime.now()))

Low Ratio Table with BNS and HFS Employee List dataframe creation completed at:  2024-09-06 18:31:27.391808


/databricks/spark/python/pyspark/sql/dataframe.py:334: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [0]:
#Calculate Lender Source, BNS/Desjardins channel and BNS Key Contact (sales manager, VP) 
sql_d = """
select 
*,
case when parent_level1_lender_cde="60002" then 
    (Case
    when lenderchannel="MOSE" and hfs_role is NULL THEN "BROKER" 
    when lenderchannel IN ("BRANCH", "HFLM", "BRM") THEN ""
    ELSE "HFA"
    end) 
end as MOSE_CHANNEL,

CASE WHEN parent_level1_lender_cde="60002" THEN 
(CASE WHEN lenderchannel="MOSE" and hfs_role is NULL then "MOSE" 
ELSE lenderchannel
END)
WHEN parent_level1_lender_cde="60001" THEN
(CASE WHEN transit_id IN ("35532", "03553") THEN "MORTGAGE SPECIALIST"
ELSE "BRANCH"
END)
WHEN parent_level1_lender_cde="20024" THEN
(CASE WHEN business_source_identification_desc_en="MORTGAGE SPECIALIST" THEN "MORTGAGE SPECIALIST" 
ELSE "BRANCH"
END) 
WHEN parent_level1_lender_cde="60003" THEN
(CASE WHEN business_source_identification_desc_en="MORTGAGE SPECIALIST" THEN "MORTGAGE SPECIALIST" 
ELSE "BRANCH"
END)
WHEN parent_level1_lender_cde="60004" THEN
(case 
when transit_id in ('27892','28072','28472','28592','28662','28672') then "MORTGAGE SPECIALIST"
when transit_id in ("73902", "91912") then "BROKER" else "BRANCH" 
END)
/*
WHEN parent_level1_lender_cde="49020" then 
(case 
when trim(TELEPHONE) ='5143764420' AND trim(business_source_identification_cde) ='13' THEN 'SPÉCIALISTE HYPOTHÉCAIRE'
when trim(TELEPHONE) ='5143764420' AND trim(business_source_identification_cde) ='04' THEN 'COURTIER'
ELSE "SUCCURSALE" 
END)

WHEN (parent_level1_lender_cde="90032" AND trim(transit_id) IN ("99048", "99051")) THEN "RMG" 
WHEN (parent_level1_lender_cde="90032" AND trim(transit_id) not IN ("99048", "99051")) THEN "MCAP"
*/
WHEN parent_level1_lender_cde IN ("90040", "90045", "90024", "90025", "20042", "30006","30106", "90049", "90044", "30085", "90043", "90050") 
THEN "BROKER"

WHEN parent_level1_lender_cde NOT IN ("60001", "60002", "60004", "49020", "90032") 
AND trim(business_source_identification_desc_en) not in ("BROKER", "MORTGAGE SPECIALIST", "BRANCH") 
then "BRANCH" 
else business_source_identification_desc_en
END AS SOURCE
from df_lr_all
 """

df_lr_1 = ss.sql(sql_d)
df_lr_1.registerTempTable("df_lr_1")


#Calculate CMHC Source based on calculated Source Channel
sql_e = """
select *,
CASE WHEN SOURCE IN ("COURTIER","BROKER", "RMG", "MCAP", "BRM") THEN "BROKER"
WHEN SOURCE = "MOSE" AND MOSE_CHANNEL="BROKER" THEN "BROKER"
WHEN SOURCE = "MOSE" AND MOSE_CHANNEL="HFA" THEN "MORTGAGE SPECIALIST"
WHEN SOURCE IN ("BRANCH", "SUCCURSALE") THEN "BRANCH"
ELSE "MORTGAGE SPECIALIST" 
END AS CMHC_SOURCE
from df_lr_1
 """

df_lr_2 = ss.sql(sql_e)
#df_lr_2.registerTempTable("df_lr_2")
#df_lr_2.write.mode("overwrite").format("orc").saveAsTable("sb_rmo_ho_reporting.test_lender_dashboard")
#print("Low Raio Table with Channel, Source, BNS Contact row count: ", df_lr_2.count())
print("Low Raio Table with Channel, Source, BNS Contact creation completed at: ", str(datetime.now()))

/databricks/spark/python/pyspark/sql/dataframe.py:334: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


Low Raio Table with Channel, Source, BNS Contact creation completed at:  2024-09-06 18:31:27.601806


In [0]:
#Calculate Low Ratio Application Messages in Last Risking - Appraisal Ruquest, PVII
vwdw_application_message_dim = pd.read_excel('vwdw_application_message_dim.xlsx').astype(str)
dfm = ss.createDataFrame(vwdw_application_message_dim)
dfm.registerTempTable("vwdw_application_message_dim")

sql_f = """
SELECT DISTINCT
lad.cmhc_loan_account_nbr,

case 
    when trim(mess.application_message_cde) IN ('298', '1203', '1207') then 'PVII Eligible'
else 'PVII Ineligible' end as PVII_last_risking,
emili.gvs_elg_ind as GVS_ind,

case 
    when lft.parent_level1_lender_cde="60002" and trim(mess.application_message_cde) ='1031' then "No Appraisal Required" 
    when lft.parent_level1_lender_cde="60002" and trim(mess.application_message_cde) in ('1032','1033')  
    then "Full Appraisal Required"
    when lft.parent_level1_lender_cde="49020" and trim(mess.application_message_cde) in ('703','706')  
    then "No Appraisal Required" 
    when lft.parent_level1_lender_cde="49020" and trim(mess.application_message_cde) in ('705','708','709','710','711')  
    then "Appraisal Required"
    when lft.parent_level1_lender_cde="49020" and trim(mess.application_message_cde) in ('704') then "Mortgage Insurance" 
    when lft.parent_level1_lender_cde="60001" and trim(mess.application_message_cde) in ('1020')  then "No Appraisal Required" 
    when lft.parent_level1_lender_cde="60001" and trim(mess.application_message_cde) in ('1021')  then "Appraisal Required"
    when lft.parent_level1_lender_cde="60003" and trim(mess.application_message_cde) ='1001' then "No Appraisal Required" 
else "No Code" end as appraisal

from cmwdw_insurance.vwdw_loan_account_dim lad

Left Join ds_ins_risk.ins_risk_last emili
    on lad.cmhc_loan_account_nbr=emili.lnacctno
INNER JOIN cmwdw_insurance.vwdw_loan_account_sum LAS
    ON (lad.loan_account_surrkey=las.loan_account_surrkey)
INNER JOIN cmwdw_insurance.vwdw_COSTABLE_UNIT_DIM CUD
    ON (las.costable_unit_surrkey=CUD.costable_unit_surrkey)
INNER JOIN cmwdw_insurance.vwdw_lender_financial_trnst_dim lfT 
    ON (las.initiation_lender_financial_transit_surrkey = lfT.lender_financial_transit_surrkey) 
INNER JOIN 
    (
    select distinct t2.cmhc_loan_account_nbr, t1.application_message_cde
    from cmwdw_insurance.tbdw_tgt_application_message_transaction_fact t2
    inner join vwdw_application_message_dim t1  
    on t1.application_message_surrkey = t2.application_message_surrkey
        inner join
            (
            select distinct 
                cmhc_loan_account_nbr,
                max(application_identifier_ts) as max_ts 
                from cmwdw_insurance.tbdw_tgt_application_message_transaction_fact t3
                inner join vwdw_application_message_dim t4
                    on t3.application_message_surrkey = t4.application_message_surrkey
                    where t4.application_message_tcd not in ('2', '3', '4') and t4.application_message_cde not in ('115')
                    group by cmhc_loan_account_nbr
                    order by cmhc_loan_account_nbr) as max
                on max.cmhc_loan_account_nbr = t2.cmhc_loan_account_nbr and max.max_ts = t2.application_identifier_ts
                where t1.application_message_cde IN ('298', '299', '1207', '1020', '1021', '1031', '1032', '1033', '703','704','706','705','708','709','710','711','1201','1202','1203', '1001')
                ) as mess
on mess.cmhc_loan_account_nbr = lad.cmhc_loan_account_nbr

WHERE
lad.application_received_system_dt >='2020-01-01'
and cud.insurance_product_group_cde = '100' 
and lad.low_ratio_ever_ind=1 /*   should it to be  lad.low_ratio_mortgage_tcd in ('01','02')    */
"""

mess = ss.sql(sql_f.format(a.get('CURRENT_YTD_BEG'), a.get('CURRENT_YTD_END'), a.get('PRIOR_YTD_BEG'), a.get('PRIOR_YTD_END'), a.get('PRIOR_YTD')))
#mess.registerTempTable("mess")
#print("Low Raio Application Message with PVII Appraisal row count: ", mess.count())
print("Low Raio Application Message with PVII Appraisal dataframe creation completed at: ", str(datetime.now()))

/databricks/spark/python/pyspark/sql/dataframe.py:334: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


Low Raio Application Message with PVII Appraisal dataframe creation completed at:  2024-09-06 18:31:28.693203


In [0]:
#Join Application messages with Low Ratio base data and save table in Sandbox
mess_2 = mess.withColumnRenamed("cmhc_loan_account_nbr","cmhc_loan_account_nbr_2")

df_result = df_lr_2.join(mess_2, df_lr_2.cmhc_loan_account_nbr == mess_2.cmhc_loan_account_nbr_2,how='left')

df_result.write.mode("overwrite").format("orc").saveAsTable("sb_rmo_ho_reporting.lender_dashboard_low_ratio")

print("Low Raio Final Data with PVII Appraisal table creation completed at: ", str(datetime.now()))

Low Raio Final Data with PVII Appraisal table creation completed at:  2024-09-06 18:34:38.797030


In [0]:
print("row count: ", df_result.count())

row count:  3520139
